In [39]:
import pandas as pd
import numpy as np

In [40]:
client = "länsförsäkringar"
df = pd.read_excel(f"../temp/{client}_results.xlsx", sheet_name=client.title())
df

,hostname,Impressions,Inviews,Fixations,Fixation/Inview Ratio,Average Fixation Duration,Total Fixation Duration,Sample Size
0,aftonbladet.se,1570562,1146130,378274,33.00,1124,425179976,medium
1,bloggar.aftonbladet.se,79122,52807,0,0.00,0,0,low
2,klart.se,4054,2308,833,36.09,2454,2044182,low
3,malservice.aftonbladet.se,5,3,1,33.33,503,503,low
4,svd.se,19702,15050,3962,26.33,4651,18427262,medium
5,tv.nu,12491,8122,2738,33.71,1234,3378692,medium


In [41]:
df = df[df["Sample Size"].isin(["high", "medium"])]
df["group"] = 1

/tmp/ipykernel_11016/497368128.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["group"] = 1


In [42]:
final_df = df.groupby(["group"]).apply(
    lambda x: pd.Series(
        {
            "Impressions": np.floor(np.average(x["Impressions"], weights=x["Impressions"])),
            "Inviews": np.floor(np.average(x["Inviews"], weights=x["Inviews"])),
            "Fixations": np.floor(np.average(x["Fixations"], weights=x["Fixations"])),
            "Fixation/Inview Ratio": round(np.average(x["Fixation/Inview Ratio"], weights=x["Impressions"]), 2),
            "Average Fixation Duration": np.floor(np.average(x["Average Fixation Duration"], weights=x["Fixations"])),
            "Total Fixation Duration": np.floor(np.average(x["Average Fixation Duration"], weights=x["Fixations"]) *  np.floor(np.average(x["Fixations"], weights=x["Fixations"])))
        }
    )
)
final_df = final_df.reset_index()
final_df

,group,Impressions,Inviews,Fixations,Fixation/Inview Ratio,Average Fixation Duration,Total Fixation Duration
0,1,1539355.0,1123667.0,371750.0,32.92,1161.0,431631797.0


In [43]:
final_df = final_df[["Impressions", "Inviews", "Fixations", "Fixation/Inview Ratio", "Average Fixation Duration", "Total Fixation Duration"]]

final_df.to_excel(f"../temp/{client}_results_consolidated.xlsx", index=False)